# Find the Minimum of $Z(x,y) = sinc(x^2+y^2)sin(2(tan^{-1}y/x))$ by Gradient Descending 

### Import necessary package.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from ipywidgets import interact

%matplotlib inline

### Prepare the meshgrid for plotting. The starting point (x<sub>0</sub>, y<sub>0</sub>) is chosen from one of the grid points using random number generator.

In [2]:
resolution = 1000
r = np.linspace(0,2.5,resolution)
theta = np.linspace(0,2.0*np.pi,resolution)

R,θ = np.meshgrid(r,theta)
Z = np.sinc(R)*np.sin(2.0*θ)
X = R*np.cos(θ); Y = R*np.sin(θ)

### Take a look at the maximum and minimum value of f(r,θ) among all the grid points.
### The analytic solution is -1 and 1.

In [3]:
print(Z.ravel().min())
print(Z.ravel().max())

-0.9999987638285974
0.9999987638285974


### Assign seed and randomly generate the staring point.

In [4]:
np.random.seed(66982445)
r_0 = r[np.random.randint(len(r))]
theta_0 = theta[np.random.randint(len(theta))]
x_0 = r_0*np.cos(theta_0)
y_0 = r_0*np.sin(theta_0)

### Print out useful information.

In [5]:
print(f'The starting point is r = {r_0}, θ = {theta_0} in polar coordinate.\nIn Cartesian coodinate,x = {x_0}, y = {y_0} .')
print(f'The value of the function as starting point is {np.sinc(r_0)*np.sin(2.0*theta_0)}')

The starting point is r = 1.2587587587587588, θ = 2.547237286694427 in polar coordinate.
In Cartesian coodinate,x = -1.0428937777018332, y = 0.704873166734761 .
The value of the function as starting point is 0.17041819009248468


### Define function to perform gradient descending.
Here we introduce a self-adjustable $\eta=1/(a+\|\nabla Z\|)$: when the gradient is large, it shrinks the step size and when gradient is small, it increase the step size, but with a upper bound $1/a$, where $a$ can be set manually.

In [6]:
def self_adj(x, factor=5.0):
    return(1.0/(factor+x))

def gradient_descending(cirteria, iter_max, r_0, theta_0):
    iteration = 0
    r_c = r_0
    theta_c = theta_0
    error = 2.*criteria
    
    if r_c==0:
        grad_r = 0.0
    else:
        grad_r = (np.cos(r_c)/r_c-np.sin(r_c)/r_c**2)*np.sin(2.0*theta_c)
    grad_theta = 2.0*np.sinc(r_c)*np.cos(2.0*theta_c)
    grad=[grad_r, grad_theta]
    grad=np.array(grad)
    norm = (grad@grad)**0.5
    η = self_adj(norm)
    
    r_record.append(str(r_c))
    theta_record.append(str(theta_c))

    while error>criteria and iteration<iter_max:
        theta_c -= grad_theta*η
        r_c -= grad_r*η
    
        r_record.append(str(r_c))
        theta_record.append(str(theta_c))

        if r_c==0:
            grad_r = 0.0;
        else:
            grad_r = (np.cos(r_c)/r_c-np.sin(r_c)/r_c**2)*np.sin(2.0*theta_c)
        grad_theta = 2.0*np.sinc(r_c)*np.cos(2.0*theta_c)
        grad = [grad_r, grad_theta]
        grad = np.array(grad)
        norm = (grad@grad)**0.5
        η = self_adj(norm)
        error = norm
        iteration += 1
        
    return(r_c, theta_c, iteration)


### We use a while loop to continue the iteration until $\|\nabla Z\|$ is smaller than our stopping criteria or the maximum iteraton times is reached. At the same time we record the coordinate (in polar coordinate) after each iteration.

In [7]:
criteria = 1e-8
iter_max = 10000
r_record = []
theta_record = []

(r_c, theta_c, iteration) = gradient_descending(criteria, iter_max, r_0, theta_0)

In [8]:
print(f'The (local) minimum is reached after {iteration} times iteration(s).')
print(f'The coordinate is r = {r_c}, θ = {theta_c} in polar coordinate;\nIn Cartesian coordinate, x = {r_c*np.cos(theta_c)}, y= {r_c*np.sin(theta_c)} .')
print(f'The minimum of the function value is obtained as Z = {np.sinc(r_c)*np.sin(2.0*theta_c)}')

The (local) minimum is reached after 261 times iteration(s).
The coordinate is r = 2.5070707727493215e-08, θ = 2.356194490192345 in polar coordinate;
In Cartesian coordinate, x = -1.772766744325643e-08, y= 1.7727667443256432e-08 .
The minimum of the function value is obtained as Z = -0.999999999999999


### Convert the polar coordinate back to Cartesian coordinate

In [9]:
for i in range(0,iteration+1):
    theta_record[i] = float(theta_record[i])
    r_record[i] = float(r_record[i])
r_record = np.array(r_record)
theta_record = np.array(theta_record)
    
x_record = r_record*np.cos(theta_record)
y_record = r_record*np.sin(theta_record)
z_record = np.sinc(r_record)*np.sin(2.0*theta_record)

### Define the plotting function

In [10]:
def plotting(index=iteration, elev=30, azim=215):
    if index>iteration:
        print('index exceed iteration times!')
    else:
        from matplotlib.ticker import LinearLocator, FormatStrFormatter

        fig = plt.figure(figsize=(12.5,10),dpi=200)
        ax = fig.gca(projection = '3d')
        surf = ax.plot_surface(X,Y,Z, cmap = cm.coolwarm, linewidth = 50, alpha=0.7)
        ax.set_zlim(-1.0,1.0)
        #ax.zaxis.set_major_locator(LinearLocator(9)) #adjust the z axis ticks
        #ax.zaxis.set_major_formatter(FormatStrFormatter('%.2f')) #adjust the digits of z axis ticks
        ax.set_zticks(np.arange(-1,1.5,0.5))
        ax.view_init(elev, azim)
    
        ax.text(x_record[0], y_record[0], z_record[0], 'Starting Point', color='r', fontsize=10)
        ax.text(x_record[index], y_record[index], z_record[index], 'End Point', color='r', fontsize=10)

        for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(12.5) # set z axis fontsize
        ax = plt.xlabel('x',fontsize=25)
        ax = plt.ylabel('y',fontsize=25)
        ax = plt.title('Find the Minimum of $sinc(x^2+y^2)sin(2tan^{-1}(y/x))$',fontsize=25,y=1.1)
        ax = plt.xticks(np.arange(-2.5,3.5,1.0),fontsize=12.5)
        ax = plt.yticks(np.arange(-2.5,3.5,1.0),fontsize=12.5)
        fig.colorbar(surf, ticks=np.arange(-1,1.2,0.2), shrink=0.4, aspect=8)
    
        ax = plt.plot(x_record[0:index], y_record[0:index], z_record[0:index], 'm.', markersize=10, alpha=1.0 )

        plt.show()

### Display by interact

In [11]:
print('Let\'s interact! Please Select the step number(index), elevation angel(elev), and azimuth angle(azim).')
interact(plotting,index=(0,iteration), elev=(-90,90), azim =(0,360));

Let's interact! Please Select the step number(index), elevation angel(elev), and azimuth angle(azim).


interactive(children=(IntSlider(value=261, description='index', max=261), IntSlider(value=30, description='ele…

### Let's play with multi-paths starting from different initial points:)

In [12]:
path = input('Enter the number of path you want:\n')
show = input('Show detail report(y/n)?\n') # See the detail including starting point, number of iteration time, and end point.

path=int(path)
np.random.seed(17588735)
r_array = r[np.random.randint(len(r), size=path)]
theta_array = theta[np.random.randint(len(theta),size=path)]
Z_array = np.zeros((path))
iteration_array = np.zeros((path)).astype(np.int32)

for i in range(0,path):
    
    r_0 = r_array[i]; theta_0 = theta_array[i];
    x_0 = r_0*np.cos(theta_0); y_0 = r_0*np.sin(theta_0)

    r_record = []
    theta_record = []
    (r_c, theta_c, iteration) = gradient_descending(criteria, iter_max, r_0, theta_0)
    Z_array[i] = np.sinc(r_c)*np.sin(2.0*theta_c)
    iteration_array[i] = iteration
    
    if show == 'y':
        print(f'Path #{i+1}:')
        print(f'\tThe starting point is r = {r_0}, θ = {theta_0} in polar coordinate.\n\tIn Cartesian coodinate,x = {x_0}, y = {y_0} .')
        print(f'\tThe value of the function as starting point is {np.sinc(r_0)*np.sin(2.0*theta_0)}')
        print('\n\tStart grdient descending:')
        print(f'\tThe (local) minimum is reached after {iteration} times iteration(s).')
        print(f'\tThe coordinate is r = {r_c}, θ = {theta_c} in polar coordinate;\n\tIn Cartesian coordinate, x = {r_c*np.cos(theta_c)}, y= {r_c*np.sin(theta_c)} .')
        print(f'\tThe minimum of the function value is obtained as Z = {Z_array[i]}')
        print('\n')

Enter the number of path you want:
500
Show detail report(y/n)?
n


### Print out the iteration time and $Z(x,y)$ and the end point for each path.

In [13]:
for i in range(0,len(Z_array)):
    print(f'Path #{i+1}: iteration time = {iteration_array[i]}, Z = {Z_array[i]:.10f}')

Path #1: iteration time = 261, Z = -1.0000000000
Path #2: iteration time = 534, Z = -1.0000000000
Path #3: iteration time = 240, Z = -1.0000000000
Path #4: iteration time = 246, Z = -1.0000000000
Path #5: iteration time = 542, Z = -1.0000000000
Path #6: iteration time = 481, Z = -1.0000000000
Path #7: iteration time = 269, Z = -1.0000000000
Path #8: iteration time = 247, Z = -1.0000000000
Path #9: iteration time = 249, Z = -1.0000000000
Path #10: iteration time = 774, Z = -1.0000000000
Path #11: iteration time = 197, Z = -1.0000000000
Path #12: iteration time = 525, Z = -1.0000000000
Path #13: iteration time = 249, Z = -1.0000000000
Path #14: iteration time = 529, Z = -1.0000000000
Path #15: iteration time = 574, Z = -1.0000000000
Path #16: iteration time = 469, Z = -1.0000000000
Path #17: iteration time = 493, Z = -1.0000000000
Path #18: iteration time = 522, Z = -1.0000000000
Path #19: iteration time = 242, Z = -1.0000000000
Path #20: iteration time = 522, Z = -1.0000000000
Path #21:

### Look at a specific path.

In [14]:
path_index = input('Enter the # of path you are interested in.\n')
path_index = int(path_index)-1

Enter the # of path you are interested in.
333


### Do gradient descending again to record the coordinate at each step along the path.

In [15]:
r_0 = r_array[path_index]; theta_0 = theta_array[path_index];
x_0 = r_0*np.cos(theta_0); y_0 = r_0*np.sin(theta_0)
criteria = 1e-10
iter_max = 10000
r_record = []
theta_record = []

(r_c, theta_c, iteration) = gradient_descending(criteria, iter_max, r_0, theta_0)

for i in range(0,iteration+1):
    theta_record[i] = float(theta_record[i])
    r_record[i] = float(r_record[i])
r_record = np.array(r_record)
theta_record = np.array(theta_record)

x_record = r_record*np.cos(theta_record)
y_record = r_record*np.sin(theta_record)
z_record = np.sinc(r_record)*np.sin(2.0*theta_record)

### Prepare the meshgrid for plotting.

In [16]:
resolution = 1000
r = np.linspace(0,r_record.max(),resolution)
theta = np.linspace(0,2.0*np.pi,resolution)

R,θ = np.meshgrid(r,theta)
Z = np.sinc(R)*np.sin(2.0*θ)
X = R*np.cos(θ); Y = R*np.sin(θ)

### Define the plotting function, which is just an extension of the previous plotting function.

In [17]:
def plotting_multi_path(index=iteration, elev=30, azim=215):
    if index>iteration:
        print('index exceed iteration times!')
    else:
        from matplotlib.ticker import LinearLocator, FormatStrFormatter

        fig = plt.figure(figsize=(12.5,10),dpi=200)
        ax = fig.gca(projection = '3d')
        
        surf = ax.plot_surface(X,Y,Z, cmap = cm.coolwarm, linewidth = 50, alpha=0.65)
        ax.set_zlim(-1.0,1.0)
        #ax.zaxis.set_major_locator(LinearLocator(9)) #adjust the z axis ticks
        #ax.zaxis.set_major_formatter(FormatStrFormatter('%.2f')) #adjust the digits of z axis ticks
        ax.set_zticks(np.arange(-1,1.5,0.5))
        ax.view_init(elev, azim)
    
        ax.text(x_record[0], y_record[0], z_record[0], 'Starting Point', color='r', fontsize=10)
        ax.text(x_record[index], y_record[index], z_record[index], 'End Point', color='r', fontsize=10)

        for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(12.5) # set z axis fontsize
        ax = plt.xlabel('x',fontsize=25)
        ax = plt.ylabel('y',fontsize=25)
        ax = plt.title('Find the Minimum of $sinc(x^2+y^2)sin(2tan^{-1}(y/x))$',fontsize=25,y=1.1)
        ax = plt.xticks(fontsize=12.5)
        ax = plt.yticks(fontsize=12.5)
        fig.colorbar(surf, ticks=np.arange(-1,1.2,0.2), shrink=0.4, aspect=8)
    
        ax = plt.plot(x_record[0:index], y_record[0:index], z_record[0:index], 'm.', markersize=10, alpha=1.0 )

        plt.show()

### Display by interact.

In [18]:
print('Let\'s interact! Please Select the step number(index), elevation angel(elev), and azimuth angle(azim).')
interact(plotting_multi_path,index=(0,iteration), elev=(-90,90), azim =(0,360));

Let's interact! Please Select the step number(index), elevation angel(elev), and azimuth angle(azim).


interactive(children=(IntSlider(value=534, description='index', max=534), IntSlider(value=30, description='ele…